In [1]:
from PIL import Image
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import albumentations as albu
#import torch
import numpy as np
from math import *
#from torch.utils.data import DataLoader
#from torch.utils.data import Dataset as BaseDataset

In [2]:
#将图片路径写成列表，为后面cv2读入方便
images_dir = 'liefeng'
ids=sorted(os.listdir(images_dir))
images_fps=[os.path.join(images_dir, image_id) for image_id in ids]

In [3]:
#批量处理，文件夹内20张照片读成np.array存入3维array‘image’，第一维代表第几张照片
j=0
image = np.zeros(shape=(len(images_fps),320,320))
for i in images_fps:
    image1 = cv2.imread(i,1)
    pil_img = Image.fromarray(cv2.cvtColor(image1,cv2.COLOR_BGR2RGB))#opencv转为pil
    #image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    #像素值变成0-1
    image2=image1[:,:,1]
    image2=image2-image2.min()
    image2=image2/image2.max()
    #照片周围变成空白像素，否则mat算法会报错
    image2[0,:]=0
    image2[image2.shape[0]-1,:]=0
    image2[:,0]=0
    image2[:,image2.shape[1]-1]=0
    image2=np.reshape(image2,(320,320))
    image[j,:,:]=image2
    j=j+1

In [4]:
#从复合图分离
'''
之前的复合图像
skeleton_img=skeleton_img*200#matplot为0-255绘图，0黑，255白。
skeleton_img_liefeng=skeleton_img+image*55
for n in range (0,19):
    #matplotlib.image.imsave("liefeng/%04.d.png"%(n+1),skeleton_img[n,:,:])
    cv2.imwrite("liefeng/%04.d.png"%(n+1),skeleton_img_liefeng[n,:,:])
'''
image_liefeng=np.zeros_like(image)
image_gujia=np.zeros_like(image)
image_gujia[image > 0.99]=1
image_liefeng[image >0.1]=1

In [5]:
Image_gujia=image_gujia*255#放大到0-255用于保存照片
Image_liefeng=image_liefeng*255#放大的到0-255用于保存照片

In [6]:
def get_gujia_point(image):
    "接受骨架灰度照片(0-1)，返回2维ndarray，记录所有骨架像素点位置"
    gujia_P=np.nonzero(image)
    gujia_p=np.array(gujia_P)
    return gujia_p
gujia_p4=get_gujia_point(image_gujia[3,:,:])

### 新增函数 get_angle（）自动识别裂缝局部图裂缝的角度，用于后续把裂缝调整到竖直用

In [113]:
#计算一张裂缝照片需要的旋转角
def get_angle(Gj_for_w):
    gj_sum_lie=np.sum(Gj_for_w,axis=0)
    gj_sum_hang=np.sum(Gj_for_w,axis=1)
    lie=np.where(gj_sum_lie!=0)
    hang=np.where(gj_sum_hang!=0)
    img_h,img_w=Gj_for_w.shape
    point1_x=np.min(lie)
    point1_y=img_h-np.where(Gj_for_w[:,point1_x]!=0)[0][0]
    point2_x=np.max(lie)
    point2_y=img_h-np.where(Gj_for_w[:,point2_x]!=0)[0][0]
    if (point2_y-point1_y)==0:
        a=90
    else:
        tan_a=(point2_x-point1_x)/(point2_y-point1_y)
        a=atan(tan_a)*180/pi
        a=int(a)
    return a

### 新增函数rotate_bound_black_bg（） 把照片逆时针旋转指定角度，且输入输出都是照片集，可以批量操作。

注：输出后的照片尺寸比原尺寸大根号2倍

In [114]:
#旋转函数
def rotate_bound_black_bg(image_batch, angle):
    # grab the dimensions of the image and then determine the,image_batch为照片numpy3维数组，第一维代表个数，23代表高和宽
    # center
    (h, w) = image_batch.shape[1:3]
    (cX, cY) = (w // 2, h // 2)
    n=image_batch.shape[0]
    #旋转后照片会变大，最大倍就是转45度时的1.414倍，math.ceil向上取整
    H=ceil(h*1.415)
    W=ceil(w*1.415)
    rotation_batch=np.zeros([n,H,W])
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    # -angle位置参数为角度参数负值表示顺时针旋转; 1.0位置参数scale是调整尺寸比例（图像缩放参数），建议0.75
    for i in range(n):
        M = cv2.getRotationMatrix2D((cX, cY), angle[i], 1.0)
        cos = np.abs(M[0, 0])
        sin = np.abs(M[0, 1])

        # compute the new bounding dimensions of the image
        nW = int((h * sin) + (w * cos))
        nH = int((h * cos) + (w * sin))

        # adjust the rotation matrix to take into account translation
        M[0, 2] += (nW / 2) - cX
        M[1, 2] += (nH / 2) - cY

        # perform the actual rotation and return the image
        # borderValue 缺失背景填充色彩，此处为白色，可自定义
        rotation=cv2.warpAffine(image_batch[i,:,:], M, (nW, nH),borderValue=(0,0,0))
        pad_h=H-rotation.shape[0]
        pad_w=W-rotation.shape[1]
        rotation=np.pad(rotation,((0,pad_h),(0,pad_w)),'constant')
        rotation_batch[i,:,:]=rotation
    return rotation_batch
    # borderValue 缺省，默认是黑色（0, 0 , 0）
    # return cv2.warpAffine(image, M, (nW, nH))
 

In [115]:
#返回一张照片的所有骨架像素点位置
def get_gujia_point(image):
    "接受骨架灰度照片(0-1)，返回2维ndarray，记录所有骨架像素点位置"
    gujia_P=np.nonzero(image)
    gujia_p=np.array(gujia_P)
    return gujia_p
gujia_p4=get_gujia_point

#返回所有框选照片
def get_img_for_w(image_liefeng,point,w,h):
    "输入image裂缝照片，point骨架点(2dimension，第一维代表纵坐标，第二位横坐标），w框选半宽，h框选半高，返回所有框选出照片,大小都相同"
    n=point.shape[1]
    image_batch=np.zeros((n,2*h+1,2*w+1))
    angles=np.zeros(n)
    for i in range(0,n-1):
        #防止-h，-w后下标越界；上界可以越没关系
        if point[0,i]-h>=0 and point[1,i]-w>=0:
            img_for_w=image_liefeng[point[0,i]-h:point[0,i]+h,point[1,i]-w:point[1,i]+w]
        elif point[0,i]-h>=0 and point[1,i]-w<0:
            img_for_w=image_liefeng[point[0,i]-h:point[0,i]+h,0:point[1,i]+w]
        elif point[0,i]-h<0 and point[1,i]-w>=0:
            img_for_w=image_liefeng[0:point[0,i]+h,point[1,i]-w:point[1,i]+w]
        else:
            img_for_w=image_liefeng[0:point[0,i]+h,0:point[1,i]+w]
        img_h,img_w=img_for_w.shape
        pad_h=2*h+1-img_h
        pad_w=2*w+1-img_w
        img_for_w=np.pad(img_for_w,((0,pad_h),(0,pad_w)),'constant')
        image_batch[i,:,:]=img_for_w
    return image_batch
#给出骨架和裂缝图片，统计点数，计算框选宽度
def calculate_width(image_for_w,gujia_for_w):
    "img_for_w框选的裂缝图，gj_for_w框选的骨架图，最后返回计算的平均宽度"
    area=np.count_nonzero(image_for_w)
    length=np.count_nonzero(gujia_for_w)
    width=area/(length)
    return width

def get_width(image_liefeng,image_gujia,w,h):
    gujia_p=get_gujia_point(image_gujia)
    width=get_w(image_liefeng,image_gujia,gujia_p,w,h)
    return width
#width=get_width(image_liefeng[1,:,:],image_gujia[1,:,:],10,10)
#width[400:450]
#img_for_w=get_img_for_w(image_liefeng[3,:,:],gujia_p4,w,h) 

### 定义get_img_and_angle_for_w（）函数
输入image裂缝照片，image骨架照片，w框选半宽，h框选半高，返回所有框选出照片和对应需要的旋转角


在get_img_for_w()函数上新增返回旋转角度数组

In [116]:
def get_img_and_angle_for_w(image_liefeng,image_gujia,w,h):
    "输入image裂缝照片，image骨架照片，w框选半宽，h框选半高，返回所有框选出照片和旋转角"
    point=get_gujia_point(image_gujia)
    n=point.shape[1]
    image_batch=np.zeros((n,2*h+1,2*w+1))
    angles=np.zeros(n)
    for i in range(0,n-1):
        #防止-h，-w后下标越界；上界可以越没关系
        if point[0,i]-h>=0 and point[1,i]-w>=0:
            img_for_w=image_liefeng[point[0,i]-h:point[0,i]+h,point[1,i]-w:point[1,i]+w]
            gj_for_w=image_gujia[point[0,i]-h:point[0,i]+h,point[1,i]-w:point[1,i]+w]
        elif point[0,i]-h>=0 and point[1,i]-w<0:
            img_for_w=image_liefeng[point[0,i]-h:point[0,i]+h,0:point[1,i]+w]
            gj_for_w=image_gujia[point[0,i]-h:point[0,i]+h,0:point[1,i]+w]
        elif point[0,i]-h<0 and point[1,i]-w>=0:
            img_for_w=image_liefeng[0:point[0,i]+h,point[1,i]-w:point[1,i]+w]
            gj_for_w=image_gujia[0:point[0,i]+h,point[1,i]-w:point[1,i]+w]
        else:
            img_for_w=image_liefeng[0:point[0,i]+h,0:point[1,i]+w]
            gj_for_w=image_gujia[0:point[0,i]+h,0:point[1,i]+w]
        img_h,img_w=img_for_w.shape
        angles[i]=get_angle(gj_for_w)
        pad_h=2*h+1-img_h
        pad_w=2*w+1-img_w
        img_for_w=np.pad(img_for_w,((0,pad_h),(0,pad_w)),'constant')
        image_batch[i,:,:]=img_for_w
    return image_batch, angles

### 用于测试get_img_and_angle_for_w
测识结果保存在‘kuangxuan_angle’文件夹中
该文件夹自己手动创建在运行目录
运行正常

In [121]:
image_for_w,angles=get_img_and_angle_for_w(image_liefeng[0,:,:],image_gujia[0,:,:],30,30)
Image_for_w=image_for_w*255
for n in range(image_for_w.shape[0]) :
    #matplotlib.image.imsave("liefeng/gujia%04.d.png"%(n+1),Image_gujia[n,:,:])
    cv2.imwrite("kuangxuan_angle/%04.d.png"%(n+1),Image_for_w[n,:,:])

### 手动创建rotation文件夹在该运行目录 为后续实验用

### 测试旋转函数rotate_bound_black_bg（），结果保存在rotation文件夹中

In [122]:
#测试用，不需运行
image_rotation=rotate_bound_black_bg(image_for_w,angles)
Image_rotation=iamge_rotation*255
for n in range(iamge_rotation.shape[0]) :
    #matplotlib.image.imsave("liefeng/gujia%04.d.png"%(n+1),Image_gujia[n,:,:])
    cv2.imwrite("rotation/%d/%04.d.png"%(0,n+1),Iamge_rotation[n,:,:])

### 所有裂缝照片（20张）得到竖直的框选，存在rotation/n/文件夹下

In [124]:
#创建用于存放照片的文件夹
pathd=os.getcwd()+'\\rotation\\'
for i in range(image_liefeng.shape[0]): 
    os.mkdir(pathd+str(i))

In [123]:
#所有裂缝照片（20张）得到竖直的框选，存在rotation/n/文件夹下,第i张裂缝的所有框选照片存在文件夹rotation/i中
for i in range(image_liefeng.shape[0]):
    image_for_w,angles=get_img_and_angle_for_w(image_liefeng[i,:,:],image_gujia[i,:,:],30,30)
    image_rotation=rotate_bound_black_bg(image_for_w,angles)
    Image_rotation=iamge_rotation*255
    for n in range(iamge_rotation.shape[0]) :
        #matplotlib.image.imsave("liefeng/gujia%04.d.png"%(n+1),Image_gujia[n,:,:])
        cv2.imwrite("rotation/%d/%04.d.png"%(i,n+1),Iamge_rotation[n,:,:])

In [181]:
#没用，get_img_and_angle_for_w可以代替
def get_img_angle(image_liefeng,image_gujia,h,w):
    "输入一张裂缝照片，可以返回所有框选照片和对应的转正的旋转角,fabs别存放再ndarray中"
    gujia_p=get_gujia_point(image_liefeng)
    img_for_w,angles=get_img_and_angle_for_w(image_liefeng,image_gujia,gujia_p,h,w)
    return img_for_w,angles

In [34]:
img_kuangxuan_2.shape[0]

231

In [39]:
#成功


In [38]:
#用于检查是否读入，cv2显式照片,0=黑，1=白
cv2.namedWindow("Image",0)
cv2.imshow('Image',image_for_w[170,:,:])
cv2.waitKey()
cv2.destroyAllWindows()

In [188]:
rotate_bound_black_bg(image_for_w,angles)

TypeError: Argument 'angle' can not be treated as a double

In [85]:
#用于检查是否读入，cv2显式照片,0=黑，1=白
cv2.namedWindow("Image",0)
cv2.imshow('Image',img_for_w[0,:,:])
cv2.waitKey()
cv2.destroyAllWindows()

(3128, 61, 61)

In [172]:
get_angle(gj_for_w[1,:,:])

-48.012787504183336

In [206]:
#用于检查是否读入，cv2显式照片,0=黑，1=白
cv2.namedWindow("Image",0)
cv2.imshow('Image',image_for_w[50,:,:])
cv2.waitKey()
cv2.destroyAllWindows()

In [191]:
print(angles[1])

38.0
